In [ ]:
import pandas as pd
import geopandas as gpd
import shapely
import numpy as np
from pathlib import Path
import os
os.environ['USE_PYGEOS'] = '0'
import rasterio
from rasterio.plot import show
from shapely.geometry import Point, Polygon
from rasterstats import zonal_stats, point_query
from rasterio.plot import show_hist
from rasterio.mask import mask
import matplotlib.pyplot as plt
import contextily as ctx
# import geemap
# import geemap.colormaps as cm

## Load data

In [ ]:
data_folder = Path('../Data/')
databases = Path('/Users/david/Dropbox/PhD/Data/')

In [ ]:
cantons_ch = gpd.read_file('/Users/david/Dropbox/PhD/Data/Databases/SITG/SHAPEFILE_LV95_LN02/swissBOUNDARIES3D_1_3_TLM_KANTONSGEBIET.shp')
cantons_ch = cantons_ch.to_crs(2056)

In [ ]:
cantons_ch[(cantons_ch.NAME =='Vaud')&(cantons_ch.KANTONSFLA.isnull()==False)].to_file('/Users/david/Dropbox/PhD/GitHub/GEOCOVID-phase2/data/canton_vd.shp', driver = "Shapefile")

In [ ]:
statpop = pd.read_csv('/Users/david/Dropbox/PhD/Data/Databases/OFS/ag-b-00.03-vz2020statpop/STATPOP2020.csv',sep = ';')
statpop_ha = statpop.copy()
geometry = [Point(xy) for xy in zip(statpop['E_KOORD'], statpop['N_KOORD'])]
statpop_point = gpd.GeoDataFrame(statpop, crs=2056, geometry=geometry)   

In [ ]:
geometry = [Polygon(zip([xy[0],xy[0],xy[0]+100,xy[0]+100],[xy[1],xy[1]+100,xy[1]+100,xy[1]])) for xy in zip(statpop_ha.E_KOORD, statpop_ha.N_KOORD)]
statpop_ha = gpd.GeoDataFrame(statpop_ha, crs=2056, geometry=geometry)      

In [ ]:
canton_vd = cantons_ch[(cantons_ch.NAME == 'Vaud')&(cantons_ch.EINWOHNERZ.isnull()==False)]
statpop_ha_4326 = statpop_ha.to_crs(4326)

## Process data

In [ ]:
stats_no2_tropospheric = pd.read_csv('/Users/david/Library/CloudStorage/GoogleDrive-dn.de.ridder@gmail.com/My Drive/AirPollution/statisticalDataOfNO2.csv')

In [ ]:
def import_raster_and_get_min_max(raster_path):
    fp = os.path.join(data_dir, raster_path)
    # Open the file:
    raster = rasterio.open(fp)

    # Check type of the variable 'raster'
    type(raster)
    # Read the raster band as separate variable
    band1 = raster.read(1)

    # Check type of the variable 'band'
#     print(type(band1))

    # Data type of the values
#     print(band1.dtype)
    # Read all bands
    array = raster.read()

    # Calculate statistics for each band
    stats = []
    for band in array:
        stats.append({
            'min': band.min(),
            'mean': band.mean(),
            'median': np.median(band),
            'max': band.max()})

    # Show stats for each channel
    return stats[0]

In [ ]:
data_dir = "/Users/david/Downloads/"
fp = os.path.join(data_dir, "Pollution NO2/download.tropospheric_NO2_column_number_density 1.tif")

In [ ]:
all_stats = []
for i in range (1,45):
    stats = import_raster_and_get_min_max("Pollution NO2/download.tropospheric_NO2_column_number_density {}.tif".format(i))
    all_stats.append(stats)

In [ ]:
stats_rasterno2 = pd.DataFrame(all_stats)

In [ ]:
stats_rasterno2 = stats_rasterno2.round(4)

In [ ]:
stats_no2_tropospheric  = stats_no2_tropospheric.round(4)

In [ ]:
pd.merge(stats_rasterno2, stats_no2_tropospheric,how = 'left', right_on = 'tropospheric_NO2_column_number_density_max', left_on = 'max') 

In [ ]:
def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [ ]:
# Open the file:
raster = rasterio.open(fp)

# Check type of the variable 'raster'
type(raster)

In [ ]:
# Read the raster band as separate variable
band1 = raster.read(1)

# Check type of the variable 'band'
print(type(band1))

# Data type of the values
print(band1.dtype)

In [ ]:
# Read all bands
array = raster.read()

# Calculate statistics for each band
stats = []
for band in array:
    stats.append({
        'min': band.min(),
        'mean': band.mean(),
        'median': np.median(band),
        'max': band.max()})

# Show stats for each channel
stats

In [ ]:
out_meta = raster.meta.copy()
print(out_meta)

In [ ]:
coords = getFeatures(cantons_ch.to_crs(4326))

out_img, out_transform = rasterio.mask.mask(dataset=raster, shapes=coords, crop=True)

In [ ]:
# Read the raster values
array = raster.read(1)

In [ ]:
show((raster, 1), cmap='terrain')

In [ ]:
# Get the affine
affine = raster.transform

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%time
no2_satellite = zonal_stats(statpop_ha_4326, array, affine=affine, stats=['mean', 'median'])

In [ ]:
no2_satellite_pt = point_query(statpop_point.to_crs(4326), array, affine=affine)

In [ ]:
statpop_ha[['NO2_tropospheric']] = pd.DataFrame(no2_satellite_pt)

In [ ]:
statpop_point_vd = gpd.sjoin(statpop_point,canton_vd[['geometry']], predicate = 'intersects').reset_index(drop  = True)
statpop_ha_vd = gpd.sjoin(statpop_ha,canton_vd[['geometry']], predicate = 'intersects').reset_index(drop  = True)

In [ ]:
statpop_ha_vd

In [ ]:
statpop_ha_vd.loc[statpop_ha_vd['no2_model_mean'] > statpop_ha_vd['no2_model_mean'].quantile(0.999), 'no2_model_mean'] = statpop_ha_vd['no2_model_mean'].mean()

In [ ]:
statpop_ha_vd[['NO2_tropospheric','geometry','B20BTOT']].explore('NO2_tropospheric')

In [ ]:
statpop_ha_vd[['NO2','geometry','B20BTOT']].explore('NO2')

In [ ]:
statpop_ha_vd[['NO2','geometry','B20BTOT']].explore('NO2')

In [ ]:
ax = canton_vd.plot(color = 'grey', figsize = (15,15))
statpop_ha_vd.plot('NO2', ax = ax, alpha = 0.6, legend = True)
ax.set_axis_off()

In [ ]:
cantons_ch.plot('mean_NO2', legend = True)

In [ ]:
statpop_ha_4326[['mean_NO2','median_NO2']] = pd.DataFrame(ns_car_day)

In [ ]:
statpop_ha_4326[statpop_ha_4326.mean_NO2.isnull()==False].plot(linewidth = 0.01)

## Attempt GEE

In [ ]:
ee.Authenticate()

ee.Initialize()

In [ ]:
# Fetch a Landsat image.
img = ee.Image('LANDSAT/LT05/C01/T1_SR/LT05_034033_20000913')

# Select Red and NIR bands, scale them, and sample 500 points.
samp_fc = img.select(['B3','B4']).divide(10000).sample(scale=30, numPixels=500)

# Arrange the sample as a list of lists.
samp_dict = samp_fc.reduceColumns(ee.Reducer.toList().repeat(2), ['B3', 'B4'])
samp_list = ee.List(samp_dict.get('list'))

# Save server-side ee.List as a client-side Python list.
samp_data = samp_list.getInfo()

# Display a scatter plot of Red-NIR sample pairs using matplotlib.
plt.scatter(samp_data[0], samp_data[1], alpha=0.2)
plt.xlabel('Red', fontsize=12)
plt.ylabel('NIR', fontsize=12)
plt.show()

In [ ]:
# Fetch a Landsat image.
NO2_OFFL = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_NO2")

In [ ]:
NO2_OFFL

### Day car noise

In [ ]:
# Data dir
data_dir = "/Users/david/Dropbox/PhD/Data/SonBaseDayCar/STRASSENLAERM_Tag"
fp = os.path.join(data_dir, "StrassenLaerm_Tag_LV95.tif")

# Open the file:
raster = rasterio.open(fp)

# Check type of the variable 'raster'
type(raster)

In [ ]:
# Read the raster band as separate variable
band1 = raster.read(1)

# Check type of the variable 'band'
print(type(band1))

# Data type of the values
print(band1.dtype)

In [ ]:
# Read all bands
array = raster.read()

# Calculate statistics for each band
stats = []
for band in array:
    stats.append({
        'min': band.min(),
        'mean': band.mean(),
        'median': np.median(band),
        'max': band.max()})

# Show stats for each channel
stats

In [ ]:
out_meta = raster.meta.copy()
print(out_meta)

In [ ]:
coords = getFeatures(cantons_ch)

out_img, out_transform = rasterio.mask.mask(dataset=raster, shapes=coords, crop=True)

In [ ]:
# Read the raster values
array = raster.read(1)

In [ ]:
show((raster, 1), cmap='terrain')

In [ ]:
# Get the affine
affine = raster.transform

In [ ]:
affine

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%%time
ns_car_day = zonal_stats(statpop_ha[['RELI','geometry']], array, affine=affine, stats=['mean', 'median'])

In [ ]:
pd.DataFrame(ns_car_day).to_pickle(data_folder/'processed/ns_car_day.pkl')

In [ ]:
ns_car_day = pd.read_pickle(data_folder/'processed/ns_car_day.pkl')

### Car Night noise

In [ ]:
# Data dir
data_dir = "/Users/david/Dropbox/PhD/Data/SonBaseNightCar/STRASSENLAERM_Nacht"
fp = os.path.join(data_dir, "StrassenLaerm_Nacht_LV95.tif")

# Open the file:
raster = rasterio.open(fp)

In [ ]:
# Check type of the variable 'raster'
type(raster)

# Read the raster band as separate variable
band1 = raster.read(1)

# Check type of the variable 'band'
print(type(band1))
# Data type of the values
print(band1.dtype)

In [ ]:
# Read all bands
array = raster.read()

# Calculate statistics for each band
stats = []
for band in array:
    stats.append({
        'min': band.min(),
        'mean': band.mean(),
        'median': np.median(band),
        'max': band.max()})

out_meta = raster.meta.copy()
print(out_meta)

In [ ]:
coords = getFeatures(cantons_ch)

out_img, out_transform = rasterio.mask.mask(dataset=raster, shapes=coords, crop=True)

In [ ]:
show((raster, 1), cmap='Gn')

# Read the raster values
array = raster.read(1)

# Get the affine
affine = raster.transform

In [ ]:
ns_car_night = zonal_stats(statpop_ha[['RELI','geometry']], array, affine=affine, stats=['mean', 'median'])

In [ ]:
pd.DataFrame(ns_car_night).to_pickle(data_folder/'processed/ns_car_night.pkl')

ns_car_night = pd.read_pickle(data_folder/'processed/ns_car_night.pkl')

In [ ]:
ns_car_day = pd.DataFrame(ns_car_day).rename(columns = {'mean':'mean_carday','median':'median_carday'})
ns_car_night = pd.DataFrame(ns_car_night).rename(columns = {'mean':'mean_carnight','median':'median_carnight'})

noise_df = pd.concat([ns_car_day,ns_car_night], axis = 1)

## Air pollution

In [ ]:
### PM10

# Data dir
fp = os.path.join(data_folder, "raw/Air pollution/pm10/pm10_2020/pm10_2020.tif")

# Open the file:
raster = rasterio.open(fp)

# Check type of the variable 'raster'
type(raster)

In [ ]:
# Read the raster values
array = raster.read(1)

In [ ]:
# Get the affine
affine = raster.transform

In [ ]:

# Find the maximum value in the band
max_value = np.max(array)
print("Maximum value in the raster is:", max_value)
band1 = np.ma.masked_equal(array, nodata)
max_value = np.max(band1)
print("Maximum value in the raster (excluding NoData values):", max_value)

In [ ]:
show((raster, 1), cmap='RdYlGn')


In [ ]:
%%time
pm10_2020_excluding_NoData = zonal_stats(statpop_ha[['RELI','geometry']], band1, affine=affine, stats=['mean', 'median'])

In [ ]:
pd.DataFrame(pm10_2020_excluding_NoData).to_pickle(data_folder/'processed/pm10_2020_excluding_NoData.pkl')

In [ ]:
pm10_2020 = pd.read_pickle(data_folder/'processed/pm10_2020.pkl')

In [ ]:
pm10_2020['mean'].mean()

In [ ]:
# Mask out NoData values if necessary
nodata = raster.nodatavals[0]  # This assumes NoData is the same for all bands
if nodata is not None:
    band1 = np.ma.masked_equal(array, nodata)

max_value = np.max(band1)
print("Maximum value in the raster (excluding NoData values):", max_value)

### PM25

In [ ]:
# Data dir
fp = os.path.join(data_folder, "raw/Air pollution/pm25/pm25_2020/pm25_2020.tif")

# Open the file:
raster = rasterio.open(fp)

# Check type of the variable 'raster'
type(raster)

In [ ]:
# Read the raster values
array = raster.read(1)

In [ ]:
# Get the affine
affine = raster.transform

In [ ]:
nodata = raster.nodatavals[0]  # This assumes NoData is the same for all bands
band1 = np.ma.masked_equal(array, nodata)

In [ ]:
%%time
pm25_2020_excluding_NoData = zonal_stats(statpop_ha[['RELI','geometry']], band1, affine=affine, stats=['mean', 'median'])

In [ ]:
pd.DataFrame(pm25_2020_excluding_NoData).to_pickle(data_folder/'processed/pm25_2020_excluding_NoData.pkl')

pm25_2020 = pd.read_pickle(data_folder/'processed/pm25_2020.pkl')

### NO2

In [ ]:
# Data dir
fp = os.path.join(data_folder, "raw/Air pollution/no2/no2_2020/no2_2020.tif")

# Open the file:
raster = rasterio.open(fp)

# Check type of the variable 'raster'
type(raster)

In [ ]:
gt = raster.transform
pixelSizeX = gt[0]
pixelSizeY =-gt[4]
print (pixelSizeX)

In [ ]:
pxsz, pysz = raster.res

In [ ]:
pysz

In [ ]:
# Read the raster values
array = raster.read(1)

In [ ]:
# Get the affine
affine = raster.transform

In [ ]:
nodata = raster.nodatavals[0]  # This assumes NoData is the same for all bands
band1 = np.ma.masked_equal(array, nodata)

In [ ]:
%%time
no2_2020_excluding_NoData = zonal_stats(statpop_ha[['RELI','geometry']], band1, affine=affine, stats=['mean', 'median'])

In [ ]:
statpop_ha[['no2_model_mean','no2_model_median']] = pd.DataFrame(no2_2020)

In [ ]:
statpop_ha['no2_model_mean'].plot.kde()

In [ ]:
pd.DataFrame(no2_2020_excluding_NoData).to_pickle(data_folder/'processed/no2_2020_excluding_NoData.pkl')

no2_2020 = pd.read_pickle(data_folder/'processed/no2_2020_excluding_NoData.pkl')

In [ ]:
# no2_2020.loc[no2_2020['mean'] > no2_2020['mean'].quantile(0.999), 'mean'] = no2_2020['mean'].mean()

In [ ]:
no2_2020['mean'].plot.kde()

## Air Pollution dataset

In [ ]:
pm10_2020 = pd.DataFrame(pm10_2020).rename(columns = {'mean':'mean_pm10','median':'median_pm10'})
pm25_2020 = pd.DataFrame(pm25_2020).rename(columns = {'mean':'mean_pm25','median':'median_pm25'})
no2_2020 = pd.DataFrame(no2_2020).rename(columns = {'mean':'mean_no2','median':'median_no2'})

In [ ]:
pollution_df = pd.concat([pm10_2020, pm25_2020, no2_2020], axis = 1)

In [ ]:
pollution_df.describe()

In [ ]:
def pct_replacement(column):
    conds = [column > np.percentile(column, 99.9)]
    choices = [np.percentile(column, 50)]
    return np.select(conds,choices,column)

In [ ]:
pollution_df = pollution_df.apply(lambda x: pct_replacement(x))

pollution_df = pd.concat([statpop_ha[['RELI','geometry']], pollution_df], axis = 1)

pollution_df = gpd.GeoDataFrame(pollution_df, crs = 2056, geometry = pollution_df['geometry'])

In [ ]:
fig, ax = plt.subplots(figsize = (20, 20))
pollution_df.plot('mean_pm10', cmap = 'magma', legend = True, ax = ax)
ctx.add_basemap(ax, source=ctx.providers.Stamen.TonerLite,crs = 'EPSG:2056')

## NDVI

In [ ]:
from os import listdir
import zipfile
import glob
from rasterio.warp import calculate_default_transform, reproject, Resampling
import re
import tarfile
from pathlib import Path

In [ ]:
# Define variables
# archive = zipfile.ZipFile('images.zip', 'r')

data_path = str("../Data/raw/Landsat 8/")
print(data_path)
bands_grep = re.compile(".*_(B2|B3|B4|B5|B10)\.TIF")
tar_list = glob.glob(data_path + '*.tar')
print(tar_list)
band_paths = []
for tar_file in tar_list:
    print(tar_file)
    # Open a connection with the first archive (in fact there's only one)
    tar = tarfile.open(tar_file)
    # Retrieve the file names from the archive
    file_list = tar.getnames()
    # Filter and keep only useful files (bands 4 and 5 to compute NDVI)
    bands = filter(lambda x: bands_grep.search(x), file_list)
    for item in bands:
        tar.extract(item, path=data_path)
        band_paths.append(data_path+item)
    # Close the connection
    tar.close()

# See what we've just extracted
print(band_paths)



In [ ]:
def load_landsat_image(img_folder, bands):
    # initialize the dictionaries
    image = {}
    ds = {}
    
    path = Path(img_folder)
    
    # loop through the given bands to load
    for band in bands:
        # considering the landsat images end with *_SR_B#.TIF, we will use it to locate the correct file
        file = next(path.glob(f'*{band}.TIF'))
        print(f'Opening file {file}')
        ds.update({band: rasterio.open(file)})
        image.update({band: ds[band].read(1)})

    return image, ds

In [ ]:
b2_ds_list = [rasterio.open(i) for i in band_paths if 'B2' in i]
b3_ds_list = [rasterio.open(i) for i in band_paths if 'B3' in i]
b4_ds_list = [rasterio.open(i) for i in band_paths if 'B4' in i]
b5_ds_list = [rasterio.open(i) for i in band_paths if 'B5' in i]
b10_ds_list = [rasterio.open(i) for i in band_paths if 'B10' in i]

In [ ]:
from rasterio.warp import reproject
dst_crs = 2056

# B2
b2_reproj_list = [reproject(source=rasterio.band(b2_ds, 1), dst_crs=dst_crs) for b2_ds in b2_ds_list]
# B3
b3_reproj_list = [reproject(source=rasterio.band(b3_ds, 1), dst_crs=dst_crs) for b3_ds in b3_ds_list]
# B4
b4_reproj_list = [reproject(source=rasterio.band(b4_ds, 1), dst_crs=dst_crs) for b4_ds in b4_ds_list]
# B5
b5_reproj_list = [reproject(source=rasterio.band(b5_ds, 1), dst_crs=dst_crs) for b5_ds in b5_ds_list]
# B10
b10_reproj_list = [reproject(source=rasterio.band(b10_ds, 1), dst_crs=dst_crs) for b10_ds in b10_ds_list]


In [ ]:
from rasterio.io import MemoryFile
from rasterio.merge import merge

def create_dataset(data, crs, transform):
    # Receives a 2D array, a transform and a crs to create a rasterio dataset
    memfile = MemoryFile()
    dataset = memfile.open(driver='GTiff', height=data.shape[0], width=data.shape[1], count=1, crs=crs, 
                           transform=transform, dtype=data.dtype)
    dataset.write(data, 1)
        
    return dataset

In [ ]:
b2_reproj_ds_list = [create_dataset(b2_reproj_ds[0][0], dst_crs, b2_reproj_ds[1]) for b2_reproj_ds in b2_reproj_list]
b3_reproj_ds_list = [create_dataset(b3_reproj_ds[0][0], dst_crs, b3_reproj_ds[1]) for b3_reproj_ds in b3_reproj_list]
b4_reproj_ds_list = [create_dataset(b4_reproj_ds[0][0], dst_crs, b4_reproj_ds[1]) for b4_reproj_ds in b4_reproj_list]
b5_reproj_ds_list = [create_dataset(b5_reproj_ds[0][0], dst_crs, b5_reproj_ds[1]) for b5_reproj_ds in b5_reproj_list]
b10_reproj_ds_list = [create_dataset(b10_reproj_ds[0][0], dst_crs, b10_reproj_ds[1]) for b10_reproj_ds in b10_reproj_list]

In [ ]:
# B2
merged_b2, transf_b2 = merge(b2_reproj_ds_list)
plt.figure(figsize=(12,6))
plt.imshow(merged_b2[0])
# B3
merged_b3, transf_b3 = merge(b3_reproj_ds_list)
plt.figure(figsize=(12,6))
plt.imshow(merged_b3[0])
# B4
merged_b4, transf_b4 = merge(b4_reproj_ds_list)
plt.figure(figsize=(12,6))
plt.imshow(merged_b4[0])
# B5
merged_b5, transf_b5 = merge(b5_reproj_ds_list)
plt.figure(figsize=(12,6))
plt.imshow(merged_b5[0])
# B10
merged_b10, transf_b10 = merge(b10_reproj_ds_list)
plt.figure(figsize=(12,6))
plt.imshow(merged_b10[0])

In [ ]:
merged_b2

In [ ]:
# Define spatial characteristics, reproject in EPSG2056
dst_crs = 'EPSG:2056'

transform, height, width = transf_b2, merged_b2[0].shape[0], merged_b2[0].shape[1]
# kwargs = src.meta.copy()
kwargs = ({'driver': 'GTiff',
    'nodata': 0.0,
    'crs': dst_crs,
    'transform': transform,
    'width': width,
    'height': height,
    'dtype': rasterio.float32,
    'count': 1
})
print(kwargs)
with rasterio.open('/Users/david/Dropbox/PhD/GitHub/Dataviz_specchio_bus/Data/B2_CH.tif', 'w', **kwargs) as dst:
        dst.write_band(1, merged_b2[0].astype(rasterio.float32))
    

transform, height, width = transf_b3, merged_b3[0].shape[0], merged_b3[0].shape[1]

with rasterio.open('/Users/david/Dropbox/PhD/GitHub/Dataviz_specchio_bus/Data/B3_CH.tif', 'w', **kwargs) as dst:
        dst.write_band(1, merged_b3[0].astype(rasterio.float32))
        

transform, height, width = transf_b4, merged_b4[0].shape[0], merged_b4[0].shape[1]

with rasterio.open('/Users/david/Dropbox/PhD/GitHub/Dataviz_specchio_bus/Data/B4_CH.tif', 'w', **kwargs) as dst:
        dst.write_band(1, merged_b4[0].astype(rasterio.float32))

In [ ]:
ndvi = (merged_b5.astype(float) - merged_b4.astype(float)) / (merged_b5.astype(float) + merged_b4.astype(float))

In [ ]:
# Define spatial characteristics, reproject in EPSG2056
dst_crs = 'EPSG:2056'

transform, height, width = transf_b4, ndvi[0].shape[0], ndvi[0].shape[1]
# kwargs = src.meta.copy()
kwargs = ({'driver': 'GTiff',
    'nodata': 0.0,
    'crs': dst_crs,
    'transform': transform,
    'width': width,
    'height': height,
    'dtype': rasterio.float32,
    'count': 1
})
print(kwargs)

In [ ]:
with rasterio.open('./Data/ndvi_CH_2056.tif', 'w', **kwargs) as dst:
        dst.write_band(1, ndvi[0].astype(rasterio.float32))

In [ ]:
stats_ndvi = zonal_stats(statpop_ha[['RELI','geometry']], './Data/ndvi_CH_2056.tif', stats=['mean', 'median', 'min', 'max'])

In [ ]:
df_nvdi = pd.DataFrame(stats_ndvi)

gdf_ndvi = gpd.GeoDataFrame(df_nvdi, crs=2056, geometry=statpop_ha['geometry'])

gdf_ndvi = gdf_ndvi.rename(columns = {'mean':'mean_ndvi','median':'median_ndvi','min':'min_ndvi','max':'max_ndvi'})

gdf_ndvi.to_pickle('./Data/gdf_ndvi.pkl')

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

In [ ]:
fig, ax = plt.subplots(figsize = (20, 20))
gdf_ndvi.plot('mean_ndvi', legend = True, ax = ax, legend_kwds = {'shrink':0.5})
ax.set_axis_off()
ax.set_title('NDVI')

## LST

In [ ]:
def calculate_LST(b10, ndvi, NDVImin, NDVImax):
    """ 1.- Calculation of TOA (Top of Atmospheric) spectral radiance.
        TOA (L) = ML * Qcal + AL
        SOURCE : https://geogeek.xyz/how-to-calculate-land-surface-temperature-with-landsat-8-images.html
        
        2.- TOA to Brightness Temperature conversion
        BT = (K2 / (ln (K1 / L) + 1)) − 273.15
        K1 = Band-specific thermal conversion constant from the metadata (K1_CONSTANT_BAND_x, where x is the thermal band number).
        K2 = Band-specific thermal conversion constant from the metadata (K2_CONSTANT_BAND_x, where x is the thermal band number).
        L = TOA
        Therefore, to obtain the results in Celsius, the radiant temperature is adjusted by adding the absolute zero (approx. -273.15°C).
        
        3.- Calculate NDVI
        Done earlier
        
        4.- Calculate the proportion of vegetation Pv
        Pv = Square ((NDVI – NDVImin) / (NDVImax – NDVImin))
        
        5.- Calculate Emissivity ε
        ε = 0.004 * Pv + 0.986
        Simply apply the formula in the raster calculator, the value of 0.986 corresponds to a correction value of the equation.
        
        6.- Calculate the Land Surface Temperature
        LST = (BT / (1 + (0.00115 * BT / 1.4388) * Ln(ε)))
        LST = Brightness Temp / 1 + Wevelenghth (10 Micron) * ( Brightness Temp / P) * Ln (e) P= hc/s = 14380 h= Plank constant c= velocity of light s = Boltzmann constant e= emissivity e=0.004Pv+0.986 Pv=proportion of vegetation= (NDVI-NDVImin / NDVImax-NDVImin)^2 LST calculation by NDVI Threshold Method (Sobrino et al, 2014)"""
    #Qcal = corresponds to band 10.
    Qcal = b10
    #ML = Band-specific multiplicative rescaling factor from the metadata (RADIANCE_MULT_BAND_x, where x is the band number).
    Ml = 3.3420E-04
    #AL = Band-specific additive rescaling factor from the metadata (RADIANCE_ADD_BAND_x, where x is the band number).
    Al = 0.1
    TOA = Ml * Qcal + Al  
    # K1 = Band-specific thermal conversion constant from the metadata (K1_CONSTANT_BAND_x, where x is the thermal band number).
    K1 = 774.8853
    # K2 = Band-specific thermal conversion constant from the metadata (K2_CONSTANT_BAND_x, where x is the thermal band number).
    K2 = 1321.0789
    BT = K2/(np.log(K1/TOA+1)) - 273.15
    
    Pv = ((ndvi - NDVImin)/(NDVImax - NDVImin))**2
    emissivity = 0.004*Pv + 0.986
    LST = (BT/(1+(0.00115*BT/1.4388)*np.log(emissivity)))
    return BT, emissivity, LST

In [ ]:
NDVImin = gdf_ndvi['mean_ndvi'].min()
NDVImax = gdf_ndvi['mean_ndvi'].max()

In [ ]:
BT, emissivity, lst = calculate_LST(merged_b10, ndvi, NDVImin, NDVImax)

In [ ]:
# Define spatial characteristics, reproject in EPSG2056
dst_crs = 'EPSG:2056'

transform, height, width = transf_b10, lst[0].shape[0], lst[0].shape[1]
# kwargs = src.meta.copy()
kwargs = ({'driver': 'GTiff',
    'nodata': 0.0,
    'crs': dst_crs,
    'transform': transform,
    'width': width,
    'height': height,
    'dtype': rasterio.float32,
    'count': 1
})
print(kwargs)

with rasterio.open('./Data/lst_ch_2056.tif', 'w', **kwargs) as dst:
        dst.write_band(1, lst[0].astype(rasterio.float32))

In [ ]:
stats_lst = zonal_stats(statpop_ha[['RELI','geometry']], './Data/lst_ch_2056.tif', stats=['mean', 'median', 'min', 'max'])

In [ ]:
df_lst = pd.DataFrame(stats_lst)

gdf_lst = gpd.GeoDataFrame(df_lst, crs=2056, geometry=statpop_ha['geometry'])

gdf_lst = gdf_lst.rename(columns = {'mean':'mean_lst','median':'median_lst','min':'min_lst','max':'max_lst'})

gdf_lst.to_pickle('./Data/gdf_lst.pkl')

In [ ]:
gdf_lst = pd.read_pickle('./Data/gdf_lst.pkl')

gdf_lst = gpd.GeoDataFrame(gdf_lst, geometry = gdf_lst['geometry'])

In [ ]:
fig, ax = plt.subplots(figsize = (20, 20))
gdf_lst.plot('mean_lst', legend = True, ax = ax, cmap = 'magma', legend_kwds = {'shrink':0.5})
ax.set_axis_off()
ax.set_title('Land Surface Temperature (LST)')

### Test : Geemap

In [ ]:
Map = geemap.Map(center=[40, -100], zoom=4)
Map

In [ ]:
dem = ee.Image('USGS/3DEP/10m')
vis = {'min': 0, 'max': 4000, 'palette': cm.palettes.dem}

In [ ]:
Map.addLayer(dem, vis, 'DEM')

In [ ]:
landcover = ee.Image("USGS/NLCD_RELEASES/2019_REL/NLCD/2019").select('landcover')

In [ ]:
Map.addLayer(landcover, {}, 'NLCD 2019')
Map.add_legend(builtin_legend='NLCD')

In [ ]:
stats = geemap.image_stats_by_zone(dem, landcover, reducer='MEAN')
stats

In [ ]:
stats.to_csv('mean.csv', index=False)

In [ ]:
geemap.image_stats_by_zone(dem, landcover, out_csv="std.csv", reducer='STD')